In [ ]:
!pip install -qU sentence_transformers openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.9/386.9 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.2/325.2 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00


In [ ]:
import json
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer

data = pd.read_excel('Path_to_file')
selected_fields = data.columns

def merge_columns(row):
    return ' '.join([f'{col}: {row[col]}' for col in data.columns])

# Apply the function to each row
data['All Informations'] = data.apply(merge_columns, axis=1)
data = data.drop(columns = selected_fields)

print(data.columns)


# Load the model
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2', device='cpu')
documents = data['All Informations'].tolist()

# Encode the documents
document_embeddings = model.encode(documents)

# Save document embeddings and original documents
with open('document_embeddings.npy', 'wb') as f:
    np.save(f, document_embeddings)

with open('documents.json', 'w') as f:
    json.dump(documents, f)

In [ ]:
data.head(10)
# document_embeddings[0]

,All Informations
0,Course name: Beginner's Guide to Python Progra...
1,Course name: Learn Machine Learning with Pytho...
2,Course name: Math & English for IBA-MBA Admiss...
3,Course name: Accelerate Your Career with Emoti...
4,Course name: A Comprehensive Study and Career ...
5,Course name: Amazon Affiliate Marketing with S...
6,Course name: Training for Innovative Teaching ...
7,Course name: Technology in Leadership Course L...
8,Course name: Speech and Language Therapy: A Pa...
9,Course name: Self-Defense with Mixed Martial A...


In [ ]:
import numpy as np
import json
from sklearn.metrics.pairwise import cosine_similarity


# Load the stored document embeddings and documents
document_embeddings = np.load('document_embeddings.npy')
with open('documents.json', 'r') as f:
    documents = json.load(f)

def retrieve_documents(query, top_k=6):
    # Encode the query
    query_embedding = model.encode([query])

    # Compute cosine similarity between the query and all document embeddings
    similarities = cosine_similarity(query_embedding, document_embeddings).flatten()

    # Get the indices of the top_k most similar documents
    top_k_indices = similarities.argsort()[-top_k:][::-1]

    # Retrieve the most similar documents
    retrieved_documents = [documents[i] for i in top_k_indices]
    return retrieved_documents


In [ ]:
# !pip install openai

In [ ]:
from openai import OpenAI
client = OpenAI(
    api_key = "API_KEY"
)
api = "API_KEY"

In [ ]:
def generate_response(query, retrieved_documents, History):
    # Combine the query and the retrieved documents into a single prompt
    prompt = f'''
    You are a career counselor chatbot. Your task is to help users achieve their career goals by suggesting a detailed plan when they express a specific career desire, and answering general career-related questions directly without suggesting a career plan otherwise. You will also given the previous conversation with the query.

    ###Step 1### Greeting
        #Greet the user and introduce yourself.
        #Ask the user how you can assist them with their career.
    ###Step 2### User Desire Recognition
        #If the user expresses a career desire, acknowledge it and proceed to Step 3.
        #If the user asks a general question, respond directly and concisely without suggesting a career plan. At first check the **Chat History** to find the relevance of the question with the previous chat if there is no Chat History then search in the **Context**.
        **Important**s: Keep track of the user's expressed career desires and conversation throughout the conversation. You will be given the **Chat History**.
    ###Step 3### Career Planning
        #Based on the user's expressed desire, suggest a detailed career plan:
        #Courses: Provide at least 3 relevant courses with links.
        #Job Categories: Suggest job categories related to the courses.
        #Salaries: Provide average salary ranges for the suggested job categories.
        #Target Companies: List potential companies where the user can apply for jobs.
        #If the user asks a follow-up question related to the previously expressed desire, respond accordingly with the relevant information.
        **Important**: Your task is to give answer based on the given **Context**.
    ###Step 4### General Questions Handling
        If the user asks general career-related questions:
            *check the **Chat History** to find the relevance of the question with the previous chat.
            *provide concise and relevant answers.
        **Ensure the response is directly related to the user's query without steering them towards a career plan unless they express a specific desire.
    ###Note###
        **If the user asks questions outside the realm of career planning (e.g., unrelated facts), reply that it is not the purpose of the chatbot.
'''

    prompt += "\n\n ###Context### ".join([f"Document {i+1}: {doc}" for i, doc in enumerate(retrieved_documents)])
    prompt += "\n\n ###Chat History### ".join([f"History {i+1}: {doc}" for i, doc in enumerate(History)])
    prompt += f"\n\nQuery: {query}"
    # Generate a response using GPT-3
    client = OpenAI(
        api_key = api#"sk-proj-nWzswFeF213sP7Hw5MybT3BlbkFJkoiBUwHJIJVMR6rfjp5j"
    )
    response = client.chat.completions.create(
        model="gpt-4o-mini",  # or any other model you have access to
        messages=[{"role": "user", "content": prompt}],
        stream=False,
        max_tokens=4096,
        n=1
    )

    return response.choices[0].message.content.strip()


In [ ]:
def HTML_generator(content):
  prompt = "Generate the following response into HTML code, and use class token to sperate the key topics: " + content
  html_response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # or any other model you have access to
        messages=[{"role": "user", "content": prompt}],
        stream=False,
        max_tokens=1024,
        n=1
    )
  return html_response.choices[0].message.content.strip()

In [ ]:
def generate_summary(content):
  prompt = "Generate the following response into summary: " + content
  html_response = client.chat.completions.create(
        model="gpt-3.5-turbo",  # or any other model you have access to
        messages=[{"role": "user", "content": prompt}],
        stream=False,
        max_tokens=1024,
        n=1
    )
  return html_response.choices[0].message.content.strip()

In [ ]:
# Example query and response generation
response = ""
History = []
total_words = 0
while True:
  query = input("Enter your query: ")
  if query == "exit":
    break

  retrieved_docs = retrieve_documents(query)
  response = generate_response(query, retrieved_docs, History)
  print()
  print(response)
  dictt = {"query": query, "response": response}
  History.append(dictt)
  total_words += sum(len(entry['query'].split()) + len(entry['response'].split()) for entry in [dictt])
  if(total_words > 3500):
    summary = generate_summary(History)
    History.clear()
    History.append({"query": "Summary til now.", "response": summary})

Enter your query: I wanna be a python developer

Hello! I'm your career counselor chatbot. It's great to hear that you want to become a Python developer! Let's work on a detailed plan to help you achieve your goal.

### Career Plan to Become a Python Developer:

#### 1. **Courses:**
   Here are some relevant courses to get you started:

   - **The Complete Python Bootcamp From Zero to Hero in Python**
     - Instructor: Jose Portilla, Pierian Training
     - Level: Beginner
     - [Course Link](https://www.udemy.com/course/complete-python-bootcamp/?couponCode=LEADERSALE24A)
   
   - **Beginner's Guide to Python Programming**
     - Instructor: Tanveer Hossain Munim
     - Level: Beginner
     - [Course Link](https://lead.academy/admin/typeahead-search?keyward=Beginner%27s%20Guide%20to%20Python%20Programming)

   - **Machine Learning with Python**
     - Instructors: Saeed Aghabozorgi, Joseph Santarcangelo
     - Level: Intermediate
     - [Course Link](https://www.coursera.org/learn/ma

In [ ]:
History

In [ ]:
for i in History:
  # print(i['response'])
  print(HTML_generator(i['response']))
  print()

In [ ]:
from flask import Flask, request, jsonify

app = Flask(__name__)

# Initialize an empty list to store the conversation history
conversation_history = []

@app.route('/chat', methods=['POST'])
def chat():
    user_input = request.json.get('user_input')

    if user_input.lower() == 'exit':
        response = "Goodbye!"
        conversation_history.clear()  # Clear history on exit
    else:
        response = f"You said: {user_input}"
        conversation_history.append({'user': user_input, 'bot': response})

    return jsonify({
        'response': response,
        'conversation_history': conversation_history
    })

if __name__ == '__main__':
    app.run(debug=True)


In [ ]:
total_words

In [ ]:
import time
st = time.time()
history = [
    {'query': 'hi', 'response': 'Hello! I am a career counselor chatbot?'},
    {'query': 'ML ML ML', 'response': 'I see you are interested in Machine Learning (ML).'}
]

# Using list comprehensions to count words
total_words = sum(len(entry['query'].split()) + len(entry['response'].split()) for entry in history)
en = time.time()
print(f'Total number of words: {total_words}', en-st)
